# Topic and word list from a working paper by Sophie Stone of the Dartmouth College Economics Department



## https://www.sbert.net/docs/usage/semantic_textual_similarity.html

In [ ]:
# Dependencies
# !pip install "calcbench-api-client[Pandas, Backoff, BeautifulSoup]" tqdm qgrid ipywidgets nltk

In [10]:
import warnings
from collections import Counter
from typing import Set, Iterable

import pandas as pd
import itertools
from tqdm.notebook import tqdm
import qgrid
import nltk

import calcbench as cb


cb.enable_backoff()
tqdm.pandas(desc="progress")

In [4]:
category_embeddings = {
    "environmental": {
        "biodiversity",
        "carbon",
        "carbon negative",
        "carbon neutral",
        "carbon zero",
        "clean",
        "clean tech",
        "clean-up",
        "climate change",
        "climate positive",
        "contamination",
        "discharge",
        "emission",
        "energy-efficient",
        "environmental",
        "environmental risk",
        "fuel",
        "fuel efficiency",
        "green",
        "greenhouse gas",
        "hazardous",
        "low carbon",
        "natural resource",
        "net-zero emission",
        "pollution",
        "remediation",
        "sustainability",
        "sustainable",
        "toxic",
        "waste",
        "water",
        "zero carbon",
        "zero net carbon",
    },
    "social": {
        "accident",
        "antiracism",
        "consumer protection",
        "customer privacy",
        "employee relation",
        "equal",
        "equal pay",
        "equity",
        "gender equality",
        "health",
        "human right",
        "justice",
        "labor relation",
        "labor standard",
        "racial awareness",
        "racial equity",
        "racial justice",
        "working condition",
    },
    "governance": {
        "advocacy",
        "antitrust",
        "board independence",
        "code of ethic",
        "compensation",
        "corporate culture",
        "corporate governance",
        "corruption",
        "governance risk",
        "justice",
        "political lobbying",
        "scandal",
        "shareholder right",
        "stability",
        "stewardship",
        "transparency",
    },
}

In [ ]:
with tqdm() as progress_bar:
    disclosures = cb.document_dataframe(
        company_identifiers=cb.tickers(index="SP500"),
        disclosure_names=["ManagementsDiscussionAndAnalysis", "RiskFactors"],
        all_history=True,
        period_type="annual",
        progress_bar=progress_bar,
        # entire_universe=True,
    )

In [ ]:
disclosures.pro

In [ ]:
def get_contents(d):
    try:
        return d.get_contents_text()
    except Exception as e:
        print(f"Exception getting {d} \n{e}")


disclosure_contents = disclosures.progress_applymap(
    get_contents, na_action="ignore"
).fillna("")

In [ ]:
disclosure_contents.to_pickle(
    r"C:\Users\andre\Dropbox (Calcbench)\andrew\sp_500_contents.pkl"
)

In [ ]:
disclosure_contents = pd.read_pickle(
    r"C:\Users\andre\Dropbox (Calcbench)\andrew\sp_500_contents.pkl"
)

In [ ]:
disclosure_embeddings = disclosure_contents.progress_applymap(
    lambda c: Counter(word.lower() for word in nltk.word_tokenize(c))
)

In [ ]:
disclosure_embeddings.to_pickle(
    r"C:\Users\andre\Dropbox (Calcbench)\andrew\sp_500_tokenize_embeddings.pkl"
)

In [5]:
disclosure_embeddings = pd.read_pickle(
    r"C:\Users\andre\Dropbox (Calcbench)\andrew\sp_500_tokenize_embeddings.pkl"
)

In [ ]:
disclosure_embeddings = disclosure_embeddings.loc[
    :, disclosure_embeddings.columns.get_level_values(1).isin(["A", "AAL"])
]

In [18]:
def distance(word_counts: Counter, category_words: Iterable[str]):
    return sum(count for word, count in word_counts.items() if word in category_words)

In [ ]:
words = category_embeddings["environmental"]

In [ ]:
counts = disclosure_embeddings.applymap(lambda c: [c[word] for word in words]).apply(
    pd.Series.explode
)

In [ ]:
counts["words"] = list(words) * disclosure_embeddings.shape[0]

In [ ]:
counts.set_index("words", append=True)

In [19]:
distances = []
for _, category_words in tqdm(category_embeddings.items()):
    category_distances = disclosure_embeddings.progress_applymap(
        lambda word_counts: distance(word_counts, category_words)
    )
    distances.append(category_distances)
esg_distances = pd.concat(
    distances, axis=1, keys=[category for category, _ in category_embeddings.items()]
)

  0%|          | 0/3 [00:00<?, ?it/s]

progress:   0%|          | 0/14940 [00:00<?, ?it/s]

progress:   0%|          | 0/14940 [00:00<?, ?it/s]

progress:   0%|          | 0/14940 [00:00<?, ?it/s]

In [20]:
esg_distances

environmental                         \
disclosure_type_name ManagementsDiscussionAndAnalysis                          
ticker                                              A  AAL AAP AAPL ABBV ABC   
period                                                                         
2007                                                0    0   0    0    0   0   
2008                                                0    0   0    0    0   0   
2009                                                3    0   0    0    0   0   
2010                                                6   76   0    0    0   0   
2011                                                6   70   1    0    0   0   
2012                                               11   67   3    0    0   0   
2013                                                9  224   2    0    1   1   
2014                                                8   70   2    0    2   1   
2015                                                6   81   3    0    2   1   
2016                                                8   90   5    0    2   1   
2017                                               10   88   2    0    2   1   
2018                                                6   95   3    0    2   5   
2019                                                6   41   1    0    2   9   
2020                                                7   24   0    0    2   8   
2021                                                0    0   0    0    0   0   

                                                                           \
disclosure_type_name                                                        
ticker               ABMD ABT ACN ADBE ADI ADM ADP ADSK  AEE  AEP AES AFL   
period                                                                      
2007                    0   0   0    0   0   0   0    0    0    0   0   0   
2008                    0   0   0    0   0   0   0    0    0    0   0   0   
2009                    0   1   0    0   6   0   0    0  161    0  70   0   
2010                    0   1   0    0   8   5   0    2  164    0  77   0   
2011                    0   1   0    0   3   4   0    2  151  123  72   0   
2012                    0   1   0    0   2   3   0    1    0    0  51   0   
2013                    0   1   0    0   1   3   0    1   64    0  26   0   
2014                    0   1   0    0   1   3   0    2   63    0  33   1   
2015                    1   1   0    0   1   1   0    2   54    0  48   1   
2016                    0   1   0    0   1   2   0    2   45    0  31   1   
2017                    0   1   0    0   1   2   1    2   40    0  38   0   
2018                    0   3   0    0   1   2   0    2   39    0  22   0   
2019                    0   3   0    0   1   3   0    1   35    0  21   0   
2020                    0   2   0    1   1   7   0    7   35    0  16   0   
2021                    0   0   0    0   0   0   0    7    0    0   0   0   

                                                                             \
disclosure_type_name                                                          
ticker                AIG AIZ AJG AKAM ALB ALGN  ALK ALL ALLE ALXN AMAT AMD   
period                                                                        
2007                    0   0   0    0   0    0    0  59    0    0    0   0   
2008                    0   0   0    0   0    0    0  61    0    0    0   0   
2009                    1   5   0    0   0    0    0  65    0    0   10   0   
2010                   66   3   0    0  34    0  131  61    0    0   13   0   
2011                    0   4  37    0  40    0   90  62    0    0   27   0   
2012                   89   5  32    0  32    0   85  61    0    0   35   2   
2013                  107   4  37    0   0    0   84  58    0    0   20   0   
2014                  126   4  40    0  32    0   81  57    3    0   15   1   
2015                  101   4  42    0  37    0   93  51    3    0   13   1   
2016                   32   

In [ ]:
esg_distances.to_excel("djia_esg_word_counts.xlsx")

In [ ]:
def matching_sentences(disclosure: str, matching_phrases: Set[str]):
    sentences = nltk.sent_tokenize(disclosure)
    for sentence in sentences:
        words = set(nltk.word_tokenize(sentence))
        matches = words & matching_phrases
        if matches:
            print(matches, sentence)

In [ ]:
disclosure_contents.applymap(
    lambda c: matching_sentences(c, category_embeddings["environmental"])
)